# AsyncIO

AsyncIO fue implementado en Python 3.4 pero no fue hasta Pythn 3.7 en que se agregaron las palabras clave ``async`` y ``await`` que veremos mas adelante, AsyncIO funciona un poco parecido a lo que es JavaScript, nos permite ejecutar tareas en un solo core (un solo hilo) simulando asincronismo con el uso del **Event Loop**.

El **event loop** es un programa que decide a que tarea ceder control para ejecutarse, es decir que el programa tiene un hilo principal el cual al completarse le dara el control al **event loop** para que verifique si hay mas tareas que se puedan ejecutar, si no las hay esperará hasta que haya alguna que este lista o se hayan realizado todas las tareas, en el siguiente grafico podremos ver mas claro la forma de actuar

![grafico](./images/img11.png)

El **event loop** se encarga de administrar y ceder el control del core o Thread principal, esto facilita al programador de las partes administrativas y de los errores y ofrece una ventaja con respecto a Threads y Process, por el lado de **Threads** nos permite manejar mucho mejor gran cantidad de tareas evitandonos problemas con el GIL mientras del lado de **Process** nos facilita y optimiza la creación de los procesos

> AsyncIO es un punto medio entre **Thread** y **Process**

Vamos a verlo en nuestro primero código

In [ ]:
import asyncio
import time


async def worker():
    print('worker - will take some time')
    time.sleep(3)
    print('worker - done it')
    return 42


async def do_something():
    print('do_something - will wait for worker')
    result = await worker()
    print('do_something - result:', result)


def main():
    print('Main - Starting')
    asyncio.run(do_something())
    print('Main - Done')


if __name__ == '__main__':
    main()


> para ver el comportamiento del código debes de ejecutar el programa ``asyncio-base.py``

y obtendremos el siguiente resultado

```python
Main - Starting
do_something - will wait for worker
worker - will take some time
worker - done it
do_something - result: 42
Main - Done
```

# Tareas con AsyncIO

nosotros podriamos reemplazar ``asyncio.run(...)`` por el uso de las palabras clave ``async`` y ``await``, ya que es la tarea mas simple y mas usada al momento de programar con este tipo estructuras, pero **asyncio** nos permite utilizar otras funciones para manipular diferentes tareas, estas son

- `asyncio.create_task()` = esta función toma una función marcada con un ``async`` y la encapsula (wrapper) en una instancia de la clase **Task** y la agrega a la cola del **Event Loop**
- `asyncio.gather(*aws)` = esta función recibe una cantidad finita de funciones async a las cuales crea una instancia Task como en el caso anterior y las agrega al **Event Loop** una vez se terminan retorna una lista con los resultados según el orden en que se enviaron cmo parámetros
- `asyncio.as_completed(aws)` = Runs each of the async functions passed to it


Hasta este punto hemos hablado de instancia la clase **Task** esta clase pone a disposición los siguiente métodos

- `cancel()` = cancela la tarea actual, al llamara este método se enviara el error ``CancelledError``
- `cancelled()` = devuelve un valor booleano si la tarea esta cancelada o no
- `result()` = devuelve el resutlado de la tarea si ya termino, en caso contrario devuelve un erro ``InvalidStateError``
- `exception()` = devuelve el error que se haya levantado en la Tarea ``CancelledError`` o ``InvalidStateError``
- `add_done_callback(callback)` = Add a callback to be run when the Task is done 
- `remove_done_callback(callback)` = Remove callback from the callbacks list

In [ ]:
import asyncio


async def worker():
    print('worker - will take some time')
    await asyncio.sleep(1)
    print('worker - Done it')
    return 42


def print_it(task):
    print('print_it result:', task.result())


async def do_something():
    print('do_something - create task for worker')
    task = asyncio.create_task(worker())
    print('do_something - add a callback')
    task.add_done_callback(print_it)
    await task
    # Information on task
    print('do_something - task.cancelled():', task.cancelled())
    print('do_something - task.done():', task.done())
    print('do_something - task.result():', task.result())
    print('do_something - task.exception():', task.exception())
    print('do_something - finished')


def main():
    print('Main - Starting')
    asyncio.run(do_something())
    print('Main - Done')


if __name__ == '__main__':
    main()


Tendremos el siguiente resultado

```python
Main - Starting
do_something - create task for worker
do_something - add a callback
worker - will take some time
worker - Done it
print_it result: 42
do_something - task.cancelled(): False
do_something - task.done(): True
do_something - task.result(): 42
do_something - task.exception(): None
do_something - finished
Main - Done
```